# Pré-processamento de dados

Neste arquivo será realizado o pré-processamento de dados, bem como filtros temporais, espaciais e visualização de dados em gráficos.

In [4]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
import numpy as np
from scipy import signal
import mne
from copy import deepcopy as dc

ModuleNotFoundError: No module named 'mne'

## Aplicação dos filtros temporais

Para a aplicação dos filtros temporais, serão utilizadas as funções a seguir:

In [ ]:
# definições de filtros

def butter_bandpass(data, lowcut, highcut, fs=256, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='bandpass')
    return signal.filtfilt(b, a, data)


def butter_lowpass(data, lowcut, fs=256, order=4):
    nyq = fs * 0.5
    low = lowcut / nyq
    b, a = signal.butter(order, low, btype='lowpass')
    return signal.filtfilt(b, a, data)


def butter_highpass(data, highcut, fs=256, order=4):
    nyq = fs * 0.5
    high = highcut / nyq
    b, a = signal.butter(order, high, btype='highpass')
    return signal.filtfilt(b, a, data)


def butter_notch(data, cutoff, var=1, fs=256, order=4):
    nyq = fs * 0.5
    low = (cutoff - var) / nyq
    high = (cutoff + var) / nyq
    print(low, high)
    b, a = signal.iirfilter(order, [low, high], btype='bandstop', ftype="butter")
    return signal.filtfilt(b, a, data, padlen=7)

## Visualização dos gráficos com e sem filtros

Power Spectrum Density - Densidade Espectral da Potência (PSD) para plotar no domínio da frequência.

PSD = |Wti|², onde o módulo é a parte real.

Notações:
- Xt: domínio do tempo
- Wf: domínio da frequência

... explicação (**tarefa** completar a explicação).

In [1]:
def print_graphs(X):
    for i in range(X.shape[1]):
        plt.plot(X[:,i])
    plt.title('Domínio do tempo')
    plt.show()

    for i in range(X.shape[1]):
        # Fs = Frequency sample (taxa de amostragem): 250Hz
        plt.psd(X[:,i], Fs=250)
    plt.title('Domínio da frequência')
    plt.show()

    for i in range(X.shape[1]):
        plt.specgram(X[i,:], Fs=250)
    plt.title('Espectrograma')
    plt.show()

In [ ]:
# Apresentação dos dados sem aplicação dos filtros

data = np.load('../dataset-s6/data.npy')

rcParams['figure.figsize'] = [17., 5.]

print_graphs(data)

## Aplicação do filtro Notch

Removendo principalmente dados de interferência da rede elétrica.

In [ ]:
data_filtered = dc(data)
data_filtered = butter_notch(data_filtered, 60)
# data_filtered = data_filtered.reshape(int(data_filtered.shape[0] / 8), 8)
print_graphs(data_filtered)

In [ ]:
# data_filtred = butter_highpass(data, 5, fs=250)
# print_graphs(data_filtred)

**Tarefa**: Executem a mesma tarefa com os outros filtros, ajustando o tempos menores (10, 20 segundos...) para visualizar melhor os dados no domínio da frequência e as suas diferenças.

### MNE:

- Criar um obj Info.
    - qtde de canais
    - taxa de amostragem
- e aplicá-lo num novo obj mne para facilita a aplicalção de métodos de filtros e vizualização dos gráficos, além de recortes de tempo nos dados.



### Teste com o objeto MNE

In [2]:
n_channels = 8
ch_types = ['eeg'] * n_channels # uma lista com 8 eeg
sfreq = 250
ch_names = ["Fp1", "Fp2", "F3", "F4", "T7", "T8", "P3", "P4"]
info = mne.create_info(ch_names=ch_names, sfreq=sfreq, ch_types=ch_types)

info.set_montage("standard_1020")
mne_data = mne.io.RawArray(data.T, info)
mne_data

SyntaxError: invalid syntax (3401879142.py, line 4)

In [ ]:
# Aplicando fft e o psd junto ao plot
# usar tmin e tmax no compute_psd para recortes
mne_data.compute_psd().plot()

In [ ]:
mne_data_filtered = mne_data.copy()
mne_data_filtered = mne_data_filtered.notch_filter(freqs=60)
mne_data_filtered = mne_data_filtered.filter(l_freq=0, h_freq=124)
mne_data_filtered.compute_psd().plot()

**Tarefa 1**: Existem parâmetros e métodos no objeto MNE, que permitem obtem janelas específicas de tempo. Utilize destas ferramentas para capturar pedaços específicos do sinal no domínio do tempo que possa ter a evocação do ritmo Beta (Utilizar o OpenBCI GUI para analisar as fatias de tempo).

**Tarefa 2**: Trabalhar em cima dos dados no domínio da frequência! Observar (para a janela de dados que possui ritmo Beta evocado por exemplo) os valores absolutos de energia (PSD). Analisar se realmente os valores que compreendem o ritmo beta está acima dos outros valores, etc. Reportar as conclusões obtidas.